# Results Analysis 

In [ ]:
# Utilities
from joblib import Parallel, delayed
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False)
from multiprocessing import Pool, cpu_count
import warnings 
warnings.filterwarnings("ignore")
import os
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
tqdm.pandas(desc="progress bar")
import gc
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# Import time packages 
import time
import datetime

# Packages needed for pre-processing:
from sklearn.preprocessing import MinMaxScaler

# Deep Learning Models 
import torch
import tensorflow as tf

# Visualization Tools
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.pyplot as plt 

# Applying Settings to Viaualization Tools 
plt.rcParams['font.size'] = 13
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
# Set color palette to blue shades
sns.set_palette(["#003366", "#6699CC", "#99CCFF", "#99CCFF"])
sns.set_palette(["#002855", "#3E7EAA", "#82B5D8", "#B3D9F2"])
plt.rcParams['font.family'] = 'Times New Roman'
sns.set_style("darkgrid")

%load_ext autoreload
%autoreload 2

In [8]:
from utils import search_folder
current_dir = os.getcwd()
thesis_folder_path = current_dir.replace("thesis_code", "")
stock_data_path = search_folder(thesis_folder_path,"stock_prices")
modelling_data_path = search_folder(thesis_folder_path,"modelling_data")
google_trending_path = search_folder(thesis_folder_path,"google_search")
results_folder = search_folder(thesis_folder_path,"results_output")

--- 

## Moderna Results 

In [122]:
moderna_results_df = pd.read_csv(results_folder + "/moderna_results_summary.csv", index_col = [0])
moderna_results_df = moderna_results_df.sort_values(by = "accuracy", ascending = False)
moderna_results_df.head()

,company,target,variables,sentiment,model,comments,precision,recall,accuracy,f1_score,mae,datetime,y_test,y_pred
trial_28,moderna,close,all,rolled_textblob_tw_sw_271,xgboost,permutations,0.527594,0.540724,0.526136,0.534078,4.849,"[Timestamp('2019-08-16 00:00:00+0000', tz='UTC...","[13.4, 14.15, 14.29, 14.16, 13.88, 14.97, 14.5...","[12.753787, 13.407628, 13.691781, 14.28641, 14..."
trial_4,moderna,close,all,rolled_textblob_tw_sw_31,xgboost,permutations,0.525843,0.529412,0.523864,0.527621,4.876,"[Timestamp('2019-08-16 00:00:00+0000', tz='UTC...","[13.4, 14.15, 14.29, 14.16, 13.88, 14.97, 14.5...","[13.10986, 12.89118, 13.87112, 14.2068205, 13...."
trial_29,moderna,close,all,rolled_textblob_tw_sw_281,xgboost,permutations,0.524775,0.527149,0.522727,0.525959,4.900,"[Timestamp('2019-08-16 00:00:00+0000', tz='UTC...","[13.4, 14.15, 14.29, 14.16, 13.88, 14.97, 14.5...","[12.753787, 13.407628, 13.691781, 14.28641, 14..."
trial_20,moderna,close,all,rolled_textblob_tw_sw_191,xgboost,permutations,0.522124,0.533937,0.520455,0.527964,4.842,"[Timestamp('2019-08-16 00:00:00+0000', tz='UTC...","[13.4, 14.15, 14.29, 14.16, 13.88, 14.97, 14.5...","[12.753787, 13.407628, 13.691781, 14.28641, 14..."
trial_23,moderna,close,all,rolled_textblob_tw_sw_221,xgboost,permutations,0.522523,0.524887,0.520455,0.523702,4.875,"[Timestamp('2019-08-16 00:00:00+0000', tz='UTC...","[13.4, 14.15, 14.29, 14.16, 13.88, 14.97, 14.5...","[12.753787, 13.407628, 13.691781, 14.28641, 14..."


In [123]:
moderna_results_df = pd.read_csv(results_folder + "/moderna_results_summary.csv", index_col = [0])
moderna_results_df.tail(1)

,company,target,variables,sentiment,model,comments,precision,recall,accuracy,f1_score,mae,datetime,y_test,y_pred
trial_38,moderna,close,all,rolled_textblob_tw_sw_371,xgboost,permutations,0.513333,0.522624,0.511364,0.517937,4.918,"[Timestamp('2019-08-16 00:00:00+0000', tz='UTC...","[13.4, 14.15, 14.29, 14.16, 13.88, 14.97, 14.5...","[12.753787, 13.407628, 13.691781, 14.28641, 14..."


In [121]:
moderna_results_df.head(1)

,company,target,variables,sentiment,model,comments,precision,recall,accuracy,f1_score,mae,datetime,y_test,y_pred
trial_0,moderna,close,all,NaN,xgboost,baseline score,0.512141,0.524887,0.510227,0.518436,4.875,"[Timestamp('2019-08-16 00:00:00+0000', tz='UTC...","[13.4, 14.15, 14.29, 14.16, 13.88, 14.97, 14.5...","[12.894571, 13.140184, 13.619936, 14.064367, 1..."
